In [1]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_diagonal_wavelet_axial_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,405642.72,-4.11,1.13,-0.93,-0.15,0.07,0.05
4,-4.11,136518.38,-1.30,-0.23,0.23,0.02,-0.01
5,1.13,-1.30,30480.31,-0.53,0.09,0.09,0.00
6,-0.93,-0.23,-0.53,13985.60,0.04,-0.03,0.00
7,-0.15,0.23,0.09,0.04,1169.98,0.00,0.00
8,0.07,0.02,0.09,-0.03,0.00,50.38,-0.00
9,0.05,-0.01,0.00,0.00,0.00,-0.00,2.15


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,-0.00002,0.00001,-0.00001,-0.00001,0.00002,0.00005
4,-0.00002,1.00000,-0.00002,-0.00001,0.00002,0.00001,-0.00002
5,0.00001,-0.00002,1.00000,-0.00003,0.00002,0.00007,0.00001
6,-0.00001,-0.00001,-0.00003,1.00000,0.00001,-0.00003,0.00002
7,-0.00001,0.00002,0.00002,0.00001,1.00000,0.00000,0.00005
8,0.00002,0.00001,0.00007,-0.00003,0.00000,1.00000,-0.00002
9,0.00005,-0.00002,0.00001,0.00002,0.00005,-0.00002,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

6.504706361215061

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.35940275e+10 2.58611877e+09 1.25657070e+08 2.40582913e+07
 1.52021332e+05 2.11797223e+02 2.64372545e-30]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.997935,-0.062806,-0.012250,-0.005596,-0.000468,-0.000020,-7.464468e-07
1,0.062028,0.996719,-0.047616,-0.020798,-0.001717,-0.000074,-3.236264e-06
2,0.014065,0.043374,0.988730,-0.142319,-0.008828,-0.000376,-1.622740e-05
3,0.008945,0.026761,0.140819,0.989188,-0.029656,-0.001267,-5.386453e-05
4,0.000962,0.002856,0.012808,0.028021,0.999287,-0.021605,-9.190042e-04
5,0.000062,0.000184,0.000821,0.001799,0.021495,0.998852,-4.275292e-02
6,0.000005,0.000016,0.000070,0.000154,0.001837,0.042723,9.990853e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.002065237749641402,
 0.0032805176082436605,
 0.011269523532281167,
 0.010812023964521855,
 0.0007132359427011181,
 0.0011475565815227684,
 0.0009147481513147593]